In [1]:
import pandas as pd
import numpy as np
from scipy import stats

# 导入数据

In [2]:
#导入整理好的汇总数据
df=pd.read_csv('../data/total.csv')
df=df[['Date','Group_Num','Code','WOR','WTR','Vol','Size','Profitability','EPR']]
df.Date=pd.to_datetime(df.Date)
display(df.head())

,Date,Group_Num,Code,WOR,WTR,Vol,Size,Profitability,EPR
0,2009-03-03,0.0,600718.SH,-0.499369,NaN,0.210875,295325.492124,0.142162,2.852842e+07
1,2009-03-03,0.0,600991.SH,-0.411673,-0.066589,0.200163,96569.256078,0.040064,1.161347e+07
2,2009-03-03,0.0,000425.SZ,-0.407139,0.017674,0.203814,589681.232985,-0.059167,-2.670190e+06
3,2009-03-03,0.0,000560.SZ,-0.278895,-0.185841,0.160781,47291.585650,0.009247,6.588908e+05
4,2009-03-03,0.0,600433.SH,-0.236562,-0.167568,0.190481,30333.600000,-0.058891,-3.167137e+06


In [3]:
#frequence: weekly -> monthly
df_m=df.groupby(['Code',pd.Grouper(key='Date',freq='M')]).first()
df_m.reset_index(inplace=True)

## 计算股票月度收益率

In [4]:
df_price=pd.read_csv('../data/论文阅读题_行情数据.csv')
#drop volume and amount 
df_price.drop(['Volume','Amount'],axis=1,inplace=True)
#set date as index which type is datetime
df_price=df_price.set_index('Date')
df_price.index=pd.to_datetime(df_price.index)
df_price.head()
#we drop data if the open price==0
df_price=df_price[df_price.Open!=0]

In [5]:
df_price.head()

,Code,PreClose,Open,Close
Date,,,,
2008-01-02,000001.SZ,38.60,38.50,37.98
2008-01-03,000001.SZ,37.98,38.00,37.35
2008-01-04,000001.SZ,37.35,37.40,38.37
2008-01-07,000001.SZ,38.37,38.18,39.17
2008-01-08,000001.SZ,39.17,39.42,40.39


In [6]:
#overght return 
df_price['CTO']=(df_price['Open']-df_price['PreClose'])/df_price['PreClose']

#计算每只股票月收益率(mr:monthly return )
df_mr=pd.DataFrame(df_price.groupby('Code').Close.resample('M').last())
df_mr['MTR']=df_mr.Close/df_mr.groupby(level=0).Close.shift(1)-1 
df_mr.reset_index(inplace=True)

# Return for the whole sample

## 定义函数

In [7]:

'''
step 1: 在adj_star_s～adj_end_s时间区间，根据df_port的CTO构建投资组合
step 2: 在adj_end_s ～period_end_s时间区间，计算投资组合的每月回报率
'''
def portfolio(adj_star_s, adj_end_s, period_end_s, df_port):
    
    #step 1
    # 计算分组当月的 MOR ( monthly average daily return ) 作为建立投资组合依据,
    df_12mMOR=df_port.groupby('Code').CTO.mean().sort_values()
    #按照 MOR分成10组,df_g中是每只股票对应的分组
    df_g=pd.DataFrame(pd.qcut(df_12mMOR, 10,labels=False))
    df_g.columns=['Port_Group']

    #step 2
    #提取在adj_end_s ～period_end_s时间区间的收益率
    df_r=df_mr[(df_mr.Date<period_end_s)&(df_mr.Date>adj_end_s)].reset_index()
    #将分组信息按照code 合并收益率上
    df_r_g=df_r.merge(df_g, on=['Code'])
    #只保留第一组和最后一组的数据
    df_r_g=df_r_g[(df_r_g.Port_Group==0)|(df_r_g.Port_Group==9)]
    #计算, equal weighted portfolio: 回报率是股票回报率的算术平均数
    r_next_6m=df_r_g.groupby(['Port_Group',pd.Grouper(key='Date')]).MTR.mean()
    
    #print(r_next_6m)
    return(r_next_6m)
    

## 计算每个月收益率

In [8]:
time_list1=pd.date_range(start='12/01/2008',end='12/1/2018', freq='6MS')
df_m_r=[]  
for i in time_list1:
    
    adj_star=i
    adj_end=adj_star+pd.DateOffset(months=1)
    adj_star_s=adj_star.strftime("%Y-%m-%d") #变成string格式输出
    adj_end_s=adj_end.strftime("%Y-%m-%d")
    period_end_s=(adj_end+pd.DateOffset(months=6)).strftime("%Y-%m-%d")
    
    #在adj_star到adj_end期间进行仓位调整，建立投资组合
    df_port=df_price[adj_star_s:adj_end_s].reset_index()
    df_m_r.append(portfolio(adj_star_s, adj_end_s, period_end_s,df_port))

In [9]:
#计算累计收益率的方程
def com_ret(x):
    return np.cumprod(x.MTR+1).tail(1)-1


In [10]:
#将分开的表整合，
df_m_r_total=pd.concat(df_m_r).reset_index().sort_values('Port_Group').reset_index(drop=True)

#平均每个月的收益率:
df_ave_mr=df_m_r_total.groupby('Port_Group').MTR.mean()

#样本期间总体收益率
df_total_return=df_m_r_total.groupby('Port_Group').apply(com_ret)

#样本期间每年收益率(1+R1)(1+R2)...(1+R12)-1
df_annual_return=pd.DataFrame(df_m_r_total.groupby(['Port_Group',pd.Grouper(key='Date',freq='Y')]).apply(com_ret))


## 结论

In [11]:
print('每个月平均收益率为\nDecile 1: ',np.round(df_ave_mr[0],6))
print('Decile10:',np.round(df_ave_mr[9],6))
print('Decile1-Decile10:',np.round(df_ave_mr[0]-df_ave_mr[9],6))

每个月平均收益率为
Decile 1:  0.007428
Decile10: 0.013443
Decile1-Decile10: -0.006015


In [12]:
print('样本期间总体收益率')
print('Decile 1: ',np.round(df_total_return[0].values[0],6))
print('Decile10:',np.round(df_total_return[9].values[0],6))
print('Decile1-Decile10:',np.round(df_total_return[0].values[0]-df_total_return[9].values[0],6))

样本期间总体收益率
Decile 1:  0.319395
Decile10: 2.160108
Decile1-Decile10: -1.840714


In [13]:
df_annual_return.reset_index(level=2,drop=True,inplace=True)
df_annual_return.reset_index(inplace=True)

In [14]:
a=df_annual_return[df_annual_return.Port_Group==0].rename(columns={'MTR':'Decile0'}).drop('Port_Group',axis=1)
b=df_annual_return[df_annual_return.Port_Group==9].rename(columns={'MTR':'Decile10'}).drop('Port_Group',axis=1)
df_an_ret=a.merge(b,on='Date')
df_an_ret['Decile1-Decile10']=df_an_ret['Decile0']-df_an_ret['Decile10']
print('组合每年收益率为')
display(df_an_ret)

组合每年收益率为


,Date,Decile0,Decile10,Decile1-Decile10
0,2009-12-31,1.285656,1.461645,-0.175989
1,2010-12-31,-0.012465,0.074072,-0.086537
2,2011-12-31,-0.422701,-0.383824,-0.038877
3,2012-12-31,-0.039917,-0.052748,0.012831
4,2013-12-31,0.076763,0.208653,-0.131891
5,2014-12-31,0.317804,0.249879,0.067925
6,2015-12-31,0.715844,1.071247,-0.355404
7,2016-12-31,-0.208145,-0.170530,-0.037615
8,2017-12-31,-0.326297,-0.154496,-0.171801
9,2018-12-31,-0.391941,-0.315299,-0.076642


原文假设如果CTO可以表征情绪，那么CTO越高，情绪越高，则公司越容易表现差。但是从实证研究来看，隔夜收益率和情绪为正相关，即高CTO高收益率，和论文结论相反，

# Return of hard-to-value subclas

在6月底进行调仓位的时候，使用3月底的Hard_to_Value指标讲过所有股票进行分组；在12分进行调仓时使用9月底的指标，这样做是为了确保在调仓时，指标是已知的。

## 定义函数

In [15]:
def subclass_by_proxy(proxy,sub_date_s,adj_star_s,adj_end_s):
    
    #提取出subdate时候的数据
    df_sub=df_m[df_m.Date==sub_date_s][['Code','Date',proxy]]
    df_sub.set_index('Code',inplace=True)
    #display(df_sub.head())

    #按照proxy分车4组，index为code，
    df_sub_g=pd.DataFrame(pd.qcut(df_sub[proxy],4,labels=False))
    df_sub_g.columns=['Proxy_Group']
    #display(df_sub_g.head()) 

    #调仓窗口:构建投资组合
    df_adj=df_price[adj_star_s:adj_end_s].reset_index()
    #第一步：proxy分组信息合并到数据上，取proxy最高和最低的两组,分别为df_low,df_high
    df_adj_sub=df_adj.merge(df_sub_g,on='Code')

    df_low=df_adj_sub[df_adj_sub.Proxy_Group==0]
    df_high=df_adj_sub[df_adj_sub.Proxy_Group==3]
    
    return [df_low,df_high]


## 计算每个proxy的月收益

In [16]:
time_list2=pd.date_range(start='6/01/2009',end='12/1/2018', freq='6MS')

In [17]:
proxy_list=['Vol','Size','Profitability','EPR']
df_proxy_r=[]
'''
df_proxy_r 为二维list，用来存放每个proxy low组和high组的月度收益率。
第一维为proxy（0:vol，1:size，2:profitability，3:EPR）
第二维为low和high组（0:low group,1:high:group)
'''
for i in proxy_list:
    proxy=i
    df_low_r=[]
    df_high_r=[]
    print('calculating monthly return for',i)
    for i in time_list2:
        adj_star=i
        adj_star_s=adj_star.strftime("%Y-%m-%d")
        adj_end_s=(adj_star+pd.DateOffset(months=1)).strftime("%Y-%m-%d")
        period_end_s=(adj_star+pd.DateOffset(months=7)).strftime("%Y-%m-%d")
        sub_date_s=(adj_star-pd.DateOffset(months=2)-pd.DateOffset(days=1)).strftime("%Y-%m-%d")
        df_port=subclass_by_proxy(proxy,sub_date_s,adj_star_s,adj_end_s)

        df_low_r.append(portfolio(adj_star_s, adj_end_s, period_end_s, df_port[0]))
        df_high_r.append(portfolio(adj_star_s, adj_end_s, period_end_s, df_port[1]))
    
    df_proxy_low=pd.concat(df_low_r).reset_index().sort_values(['Port_Group','Date'])
    df_proxy_high=pd.concat(df_high_r).reset_index().sort_values(['Port_Group','Date'])
    
    df_proxy_r.append([df_proxy_low,df_proxy_high])

calculating monthly return for Vol
calculating monthly return for Size
calculating monthly return for Profitability
calculating monthly return for EPR


## 结论

In [18]:
index_list=['Vol_low','Vol_high','Size_low','Size_high','Profit_low','Profit_high','EPR_low','EPR_high']

### 每个月平均收益率

In [19]:
l_ave_mr=[]
for i in range(4):
    for j in range(2):
        a=df_proxy_r[i][j].groupby('Port_Group').mean().values.reshape(2,)
        l_ave_mr.append(a)
        

In [20]:
df_ave_mr=pd.DataFrame(l_ave_mr)
df_ave_mr.columns=['Decile1','Decile10']
df_ave_mr['Decile1-Decile10']=df_ave_mr['Decile1']-df_ave_mr['Decile10']
df_ave_mr.index=index_list
print('每个proxy平均月度收益率')
display(df_ave_mr)

每个proxy平均月度收益率


,Decile1,Decile10,Decile1-Decile10
Vol_low,0.006908,0.009140,-0.002231
Vol_high,0.001434,0.005790,-0.004355
Size_low,0.008316,0.011024,-0.002709
Size_high,-0.001440,0.002392,-0.003832
Profit_low,0.010045,0.008883,0.001162
Profit_high,0.002373,0.002554,-0.000181
EPR_low,0.008767,0.009090,-0.000323
EPR_high,0.001500,0.003020,-0.001520


**1.  从Decile1-Decile10可以看出除了profit_low组，其他7个小组都是隔夜收益率高，收益率高**

**2.  Decile1-Decile10的差值绝对值在波动率高、市值高、利润低、EPR高的时候较大**


### 样本期间累积收益率

In [21]:
l_cum_mr=[]
for i in range(4):
    for j in range(2):
        a=df_proxy_r[i][j].groupby('Port_Group').apply(com_ret).values
        l_cum_mr.append(a)

In [22]:
df_cum_mr=pd.DataFrame(l_cum_mr)
df_cum_mr.columns=['Decile1','Decile10']
df_cum_mr['Decile1-Decile10']=df_cum_mr['Decile1']-df_cum_mr['Decile10']
df_cum_mr.index=index_list
print('每个proxy平均月度收益率')
display(df_cum_mr)

每个proxy平均月度收益率


,Decile1,Decile10,Decile1-Decile10
Vol_low,0.387841,0.942385,-0.554544
Vol_high,-0.408466,0.096017,-0.504483
Size_low,0.402850,1.052647,-0.649798
Size_high,-0.468342,-0.144094,-0.324248
Profit_low,0.718880,0.622425,0.096455
Profit_high,-0.196986,-0.116700,-0.080286
EPR_low,0.477295,0.655909,-0.178615
EPR_high,-0.234782,-0.054016,-0.180765


Vol_low： 波动率

### 每年收益率

In [23]:
l_ann_mr=[]
for i in range(4):
    for j in range(2):
        a=pd.DataFrame(df_proxy_r[0][1].groupby(['Port_Group',pd.Grouper(key='Date',freq='Y')]).apply(com_ret).reset_index(level=2,drop=True))
        l_ann_mr.append(a)

In [24]:
pd_ann_mr=pd.concat(l_ann_mr,axis=1)
pd_ann_mr.columns=index_list
pd_ann_mr

Vol_low  Vol_high  Size_low  Size_high  Profit_low  \
Port_Group Date                                                              
0          2009-12-31  0.175199  0.175199  0.175199   0.175199    0.175199   
           2010-12-31 -0.006633 -0.006633 -0.006633  -0.006633   -0.006633   
           2011-12-31 -0.384850 -0.384850 -0.384850  -0.384850   -0.384850   
           2012-12-31 -0.043490 -0.043490 -0.043490  -0.043490   -0.043490   
           2013-12-31  0.009496  0.009496  0.009496   0.009496    0.009496   
           2014-12-31  0.252223  0.252223  0.252223   0.252223    0.252223   
           2015-12-31  0.662258  0.662258  0.662258   0.662258    0.662258   
           2016-12-31 -0.243586 -0.243586 -0.243586  -0.243586   -0.243586   
           2017-12-31 -0.313022 -0.313022 -0.313022  -0.313022   -0.313022   
           2018-12-31 -0.416961 -0.416961 -0.416961  -0.416961   -0.416961   
           2019-12-31  0.352707  0.352707  0.352707   0.352707    0.352707   
9          2009-12-31  0.257383  0.257383  0.257383   0.257383    0.257383   
           2010-12-31  0.165402  0.165402  0.165402   0.165402    0.165402   
           2011-12-31 -0.338033 -0.338033 -0.338033  -0.338033   -0.338033   
           2012-12-31 -0.097942 -0.097942 -0.097942  -0.097942   -0.097942   
           2013-12-31  0.238359  0.238359  0.238359   0.238359    0.238359   
           2014-12-31  0.255364  0.255364  0.255364   0.255364    0.255364   
           2015-12-31  0.791827  0.791827  0.791827   0.791827    0.791827   
           2016-12-31 -0.370785 -0.370785 -0.370785  -0.370785   -0.370785   
           2017-12-31 -0.222854 -0.222854 -0.222854  -0.222854   -0.222854   
           2018-12-31 -0.316322 -0.316322 -0.316322  -0.316322   -0.316322   
           2019-12-31  0.345048  0.345048  0.345048   0.345048    0.345048   

                       Profit_high   EPR_low  EPR_high  
Port_Group Date                                         
0          2009-12-31     0.175199  0.175199  0.175199  
           2010-12-31    -0.006633 -0.006633 -0.006633  
           2011-12-31    -0.384850 -0.384850 -0.384850  
           2012-12-31    -0.043490 -0.043490 -0.043490  
           2013-12-31     0.009496  0.009496  0.009496  
           2014-12-31     0.252223  0.252223  0.252223  
           2015-12-31     0.662258  0.662258  0.662258  
           2016-12-31    -0.243586 -0.243586 -0.243586  
           2017-12-31    -0.313022 -0.313022 -0.313022  
           2018-12-31    -0.416961 -0.416961 -0.416961  
           2019-12-31     0.352707  0.352707  0.352707  
9          2009-12-31     0.257383  0.257383  0.257383  
           2010-12-31     0.165402  0.165402  0.165402  
           2011-12-31    -0.338033 -0.338033 -0.338033  
           2012-12-31    -0.097942 -0.097942 -0.097942  
           2013-12-31     0.238359  0.238359  0.238359  
           2014-12-31     0.255364  0.255364  0.255364  
           2015-12-31     0.791827  0.791827  0.791827  
           2016-12-31    -0.370785 -0.370785 -0.370785  
           2017-12-31    -0.222854 -0.222854 -0.222854  
           2018-12-31    -0.316322 -0.316322 -0.316322  
           2019-12-31     0.345048  0.345048  0.345048